In [202]:
# import libraries 
import pandas as pd 
import folium
import json

In [203]:
#혐오범죄 파일 읽어오기
df = pd.read_excel('./hate_crime_dataset.xlsx')

# 주별로 구분이 가능하도록 개별 주 경계 데이터로 나눠준다

In [218]:
total_boundary_path = 'dataset/boundary.json'

# 전체 주별 json파일 열기
with open(total_boundary_path) as f:
    data = json.load(f)

# 개별 주 경계 데이터를 담을 딕셔너리
states_data = {}

count = 0
# GeoJSON 데이터를 순회하며 주(State) 경계 데이터 분리
for feature in data['features']:
    state_name = feature['properties']['NAME']
    states_data[state_name] = {
        'type': 'FeatureCollection',
        'features': [feature]
    }

# 개별 주 경계 데이터를 개별 파일로 저장
for state_name, state_data in states_data.items():
    state_json_path = f'state_boundary_data/{state_name}.json'
    
    with open(state_json_path, 'w') as f:
        json.dump(state_data, f)

In [219]:
#Federal이 state_name인 경우는 연방 수사국이 자체적으로 조사한 혐오범죄로 특정 주로 인식하지 못하여 제거해준다
df = df[df["state_name"] != "Federal"]
df["state_name"].unique()

array(['Alaska', 'Alabama', 'Arkansas', 'Arizona', 'California',
       'Colorado', 'Connecticut', 'District of Columbia', 'Delaware',
       'Florida', 'Georgia', 'Hawaii', 'Iowa', 'Idaho', 'Illinois',
       'Indiana', 'Kansas', 'Kentucky', 'Louisiana', 'Massachusetts',
       'Maryland', 'Maine', 'Michigan', 'Minnesota', 'Missouri',
       'Mississippi', 'Montana', 'Nebraska', 'North Carolina',
       'North Dakota', 'New Hampshire', 'New Jersey', 'New Mexico',
       'Nevada', 'New York', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Virginia', 'Vermont', 'Washington', 'Wisconsin',
       'West Virginia', 'Wyoming'], dtype=object)

In [220]:
normalized_data = []
#2017~2021년도의 정규화 데이터 저장
for i in [2017,2018,2019,2020,2021]:
    normalized_data.append(pd.read_csv(f"dataset/normalized_dataset_{i}.csv"))

#number_of_offenders_normalized는 공격자가 특정되지 않은 경우 0이될 수 있어서 hate_crime_case_normalized를 이용하는 것이 가장 효과적

In [221]:
def style_function(feature):
    return {
        'fillColor': 'gray',
        'weight': 1,
    }

# 단순 사건 수를 이용한 시각화

In [222]:
for i in [2017,2018,2019,2020,2021]:
    #2017 ~ 2021년도 데이터 순차적 출력
    data = normalized_data[i-2017].set_index('state_name')['hate_crime_case_normalized'].squeeze()
    
    #8개의 구역으로 경계를 나눔
    bins = list(data.quantile([0,0.15,0.3,0.45,0.60,0.75,0.9,1]))

    # Folium 지도 객체 생성
    m = folium.Map(
        location=[38, -95],
        zoom_start=5,
        tiles='cartodbpositron',
    )  # 초기 지도 중앙 위치와 확대 레벨 타일 모양 설정

    m.choropleth(
        geo_data= total_boundary_path,
        #2017 ~ 2021년도 데이터 순차적 출력
        data=data,
        fill_color='YlOrRd',
        fill_opacity=0.8,
        line_opacity=0.4,
        key_on='properties.NAME',
        legend_name=f"{i}년도 주별 혐오범죄 발생",
        bins=bins,
        )
    # JSON 파일을 Folium에 추가
    for state in df["state_name"].unique():
        geojson_path = f'state_boundary_data/{state}.json'  # 각 주의 경계 데이터 파일 경로
        folium.GeoJson(
            geojson_path,
            name=state,
            style_function=style_function,
        ).add_to(m)

    folium.LayerControl().add_to(m)
    # 지도를 HTML 파일로 저장
    m.save(f'Total_map/{i}_map.html')

C:\Users\parkg\anaconda3\envs\2023-DS-B\lib\site-packages\folium\folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(
C:\Users\parkg\anaconda3\envs\2023-DS-B\lib\site-packages\folium\folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(
C:\Users\parkg\anaconda3\envs\2023-DS-B\lib\site-packages\folium\folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(
C:\Users\parkg\anaconda3\envs\2023-DS-B\lib\site-packages\folium\folium.py:465: FutureWarning: The choropleth  method 

Alabama주에는 거의 혐오범죄가 없었는데 2021년도에 크게 증가한것이 좀 신기하다

# 2017 ~ 2021 통합자료 시각화

In [223]:
intergrated_data = normalized_data[0]
for i in range(1,5):
    intergrated_data['hate_crime_case_normalized'] += normalized_data[i]['hate_crime_case_normalized']

In [224]:
data = intergrated_data.set_index('state_name')['hate_crime_case_normalized'].squeeze()
    
#8개의 구역으로 경계를 나눔
bins = list(data.quantile([0,0.15,0.3,0.45,0.60,0.75,0.9,1]))

# Folium 지도 객체 생성
m = folium.Map(
    location=[38, -95],
    zoom_start=5,
    tiles='cartodbpositron',
)  # 초기 지도 중앙 위치와 확대 레벨 타일 모양 설정

m.choropleth(
    geo_data= total_boundary_path,
    #2017 ~ 2021년도 데이터 순차적 출력
    data=data,
    fill_color='YlOrRd',
    fill_opacity=0.8,
    line_opacity=0.4,
    key_on='properties.NAME',
    legend_name=f"2017~2021년도 통합 주별 혐오범죄 발생",
    bins=bins,
    )
# JSON 파일을 Folium에 추가
for state in df["state_name"].unique():
    geojson_path = f'state_boundary_data/{state}.json'  # 각 주의 경계 데이터 파일 경로
    folium.GeoJson(
        geojson_path,
        name=state,
        style_function=style_function,
    ).add_to(m)

folium.LayerControl().add_to(m)
# 지도를 HTML 파일로 저장
m.save(f'intergrated_map/2017~2021_map.html')

C:\Users\parkg\anaconda3\envs\2023-DS-B\lib\site-packages\folium\folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [225]:
normalized_race_data = []
#2017~2021년도의 정규화 데이터 저장
for i in [2017,2018,2019,2020,2021]:
    normalized_race_data.append(pd.read_csv(f"dataset/norm_{i}_race.csv"))

# 인종을 구분한 시각화 - 아시안 인종 적용

In [226]:
for i in [2017,2018,2019,2020,2021]:
    #2017 ~ 2021년도 데이터 순차적 출력
    #주별 아시안 인종에 대할 차별 추출
    data = normalized_race_data[i-2017].set_index('State')['Asian'].squeeze()
    
    #8개의 구역으로 경계를 나눔
    bins = list(data.quantile([0,0.15,0.3,0.45,0.60,0.75,0.9,1]))

    # Folium 지도 객체 생성
    m = folium.Map(
        location=[38, -95],
        zoom_start=5,
        tiles='cartodbpositron',
    )  # 초기 지도 중앙 위치와 확대 레벨 타일 모양 설정

    m.choropleth(
        geo_data= total_boundary_path,
        #2017 ~ 2021년도 데이터 순차적 출력
        data=data,
        fill_color='YlOrRd',
        fill_opacity=0.8,
        line_opacity=0.4,
        key_on='properties.NAME',
        legend_name=f"{i}년도 주별 아시안 인종 혐오범죄 발생",
        bins=bins,
        )
    # JSON 파일을 Folium에 추가
    for state in df["state_name"].unique():
        geojson_path = f'state_boundary_data/{state}.json'  # 각 주의 경계 데이터 파일 경로
        folium.GeoJson(
            geojson_path,
            name=state,
            style_function=style_function,
        ).add_to(m)

    folium.LayerControl().add_to(m)
    # 지도를 HTML 파일로 저장
    m.save(f'Asian_map/{i}_Asian_hate_map.html')

C:\Users\parkg\anaconda3\envs\2023-DS-B\lib\site-packages\folium\folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(
C:\Users\parkg\anaconda3\envs\2023-DS-B\lib\site-packages\folium\folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(
C:\Users\parkg\anaconda3\envs\2023-DS-B\lib\site-packages\folium\folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(
C:\Users\parkg\anaconda3\envs\2023-DS-B\lib\site-packages\folium\folium.py:465: FutureWarning: The choropleth  method 

# 인종을 구분한 시각화 - 흑인 적용

In [227]:
for i in [2017,2018,2019,2020,2021]:
    #2017 ~ 2021년도 데이터 순차적 출력
    #주별 아시안 인종에 대할 차별 추출
    data = normalized_race_data[i-2017].set_index('State')['Asian'].squeeze()
    
    #8개의 구역으로 경계를 나눔
    bins = list(data.quantile([0,0.15,0.3,0.45,0.60,0.75,0.9,1]))

    # Folium 지도 객체 생성
    m = folium.Map(
        location=[38, -95],
        zoom_start=5,
        tiles='cartodbpositron',
    )  # 초기 지도 중앙 위치와 확대 레벨 타일 모양 설정

    m.choropleth(
        geo_data= total_boundary_path,
        #2017 ~ 2021년도 데이터 순차적 출력
        data=data,
        fill_color='YlOrRd',
        fill_opacity=0.8,
        line_opacity=0.4,
        key_on='properties.NAME',
        legend_name=f"{i}년도 주별 흑인 혐오범죄 발생",
        bins=bins,
        )
    # JSON 파일을 Folium에 추가
    for state in df["state_name"].unique():
        geojson_path = f'state_boundary_data/{state}.json'  # 각 주의 경계 데이터 파일 경로
        folium.GeoJson(
            geojson_path,
            name=state,
            style_function=style_function,
        ).add_to(m)

    folium.LayerControl().add_to(m)
    # 지도를 HTML 파일로 저장
    m.save(f'Black_map/{i}_Asian_hate_map.html')

C:\Users\parkg\anaconda3\envs\2023-DS-B\lib\site-packages\folium\folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(
C:\Users\parkg\anaconda3\envs\2023-DS-B\lib\site-packages\folium\folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(
C:\Users\parkg\anaconda3\envs\2023-DS-B\lib\site-packages\folium\folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(
C:\Users\parkg\anaconda3\envs\2023-DS-B\lib\site-packages\folium\folium.py:465: FutureWarning: The choropleth  method 

In [228]:
intergrated_data = normalized_race_data[0].set_index('State')['Asian'].squeeze()

for i in range(1,5):
    intergrated_data += normalized_race_data[i].set_index('State')['Asian'].squeeze()

# 인종을 구분한 통합 시각화 - 아시안 인종 적용

In [229]:
data = intergrated_data
    
#8개의 구역으로 경계를 나눔
bins = list(data.quantile([0,0.15,0.3,0.45,0.60,0.75,0.9,1]))

# Folium 지도 객체 생성
m = folium.Map(
    location=[38, -95],
    zoom_start=5,
    tiles='cartodbpositron',
)  # 초기 지도 중앙 위치와 확대 레벨 타일 모양 설정

m.choropleth(
    geo_data= total_boundary_path,
    #2017 ~ 2021년도 데이터 순차적 출력
    data=data,
    fill_color='YlOrRd',
    fill_opacity=0.8,
    line_opacity=0.4,
    key_on='properties.NAME',
    legend_name=f"주별 2017~2021 아시안 인종 혐오범죄 발생",
    bins=bins,
    )
# JSON 파일을 Folium에 추가
for state in df["state_name"].unique():
    geojson_path = f'state_boundary_data/{state}.json'  # 각 주의 경계 데이터 파일 경로
    folium.GeoJson(
        geojson_path,
        name=state,
        style_function=style_function,
    ).add_to(m)

folium.LayerControl().add_to(m)
# 지도를 HTML 파일로 저장
m.save(f'intergrated_map/intergrated_Asian_hate_map.html')

C:\Users\parkg\anaconda3\envs\2023-DS-B\lib\site-packages\folium\folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [230]:
intergrated_data = normalized_race_data[0].set_index('State')['Black'].squeeze()

for i in range(1,5):
    intergrated_data += normalized_race_data[i].set_index('State')['Black'].squeeze()

# 인종을 구분한 통합 시각화 - 흑인 적용

In [231]:
data = intergrated_data
    
#8개의 구역으로 경계를 나눔
bins = list(data.quantile([0,0.15,0.3,0.45,0.60,0.75,0.9,1]))

# Folium 지도 객체 생성
m = folium.Map(
    location=[38, -95],
    zoom_start=5,
    tiles='cartodbpositron',
)  # 초기 지도 중앙 위치와 확대 레벨 타일 모양 설정

m.choropleth(
    geo_data= total_boundary_path,
    #2017 ~ 2021년도 데이터 순차적 출력
    data=data,
    fill_color='YlOrRd',
    fill_opacity=0.8,
    line_opacity=0.4,
    key_on='properties.NAME',
    legend_name=f"주별 2017~2021 흑인 혐오범죄 발생",
    bins=bins,
    )
# JSON 파일을 Folium에 추가
for state in df["state_name"].unique():
    geojson_path = f'state_boundary_data/{state}.json'  # 각 주의 경계 데이터 파일 경로
    folium.GeoJson(
        geojson_path,
        name=state,
        style_function=style_function,
    ).add_to(m)

folium.LayerControl().add_to(m)
# 지도를 HTML 파일로 저장
m.save(f'intergrated_map/intergrated_Black_hate_map.html')

C:\Users\parkg\anaconda3\envs\2023-DS-B\lib\site-packages\folium\folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(
